## Import Dependencies

In [2]:
import petpy
from petpy import Petfinder
from config import API_key, API_secret, mongo_conn
import json
import pymongo
import pandas as pd
from pprint import pprint

## API Call and Pandas DF Processing

In [3]:
pf = Petfinder(key=API_key, secret=API_secret)

In [4]:
# animals = pf.animals() - see below for query with edited results limits
animal_df = pf.animals(results_per_page=100, pages=100, return_df=True)

/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/petpy/api.py:1049: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  results_df = json_normalize(results[key])


In [5]:
animal2_df = animal_df.copy()

In [6]:
# get rid of extra organization_id column
my_columns = list(range(0,48))
animal2_df = animal2_df.iloc[:,my_columns]

In [7]:
# inspect list for column name formats
column_list = list(animal2_df.columns) 
column_list

['id',
 'organization_id',
 'url',
 'type',
 'species',
 'age',
 'gender',
 'size',
 'coat',
 'tags',
 'name',
 'description',
 'organization_animal_id',
 'photos',
 'videos',
 'status',
 'status_changed_at',
 'published_at',
 'distance',
 'breeds.primary',
 'breeds.secondary',
 'breeds.mixed',
 'breeds.unknown',
 'colors.primary',
 'colors.secondary',
 'colors.tertiary',
 'attributes.spayed_neutered',
 'attributes.house_trained',
 'attributes.declawed',
 'attributes.special_needs',
 'attributes.shots_current',
 'environment.children',
 'environment.dogs',
 'environment.cats',
 'primary_photo_cropped.small',
 'primary_photo_cropped.medium',
 'primary_photo_cropped.large',
 'primary_photo_cropped.full',
 'contact.email',
 'contact.phone',
 'contact.address.address1',
 'contact.address.address2',
 'contact.address.city',
 'contact.address.state',
 'contact.address.postcode',
 'contact.address.country',
 'animal_id',
 'animal_type']

In [8]:
# change column header names in animal2_df to remove dot notation for mongo
animal3_df = animal2_df.rename(columns = {
 'breeds.primary':'breeds_primary',
 'breeds.secondary': 'breeds_secondary',
 'breeds.mixed': 'breeds_mixed',
 'breeds.unknown':'breeds_unknown',
 'colors.primary': 'colors_primary',
 'colors.secondary': 'colors_secondary',
 'colors.tertiary': 'colors_tertiary',
 'attributes.spayed_neutered': 'attributes_spayed_neutered',
 'attributes.house_trained': 'attributes_house_trained',
 'attributes.declawed': 'attributes_declawed',
 'attributes.special_needs': 'attributes_special_needs',
 'attributes.shots_current': 'attributes_shots_current',
 'environment.children': 'environment_children',
 'environment.dogs': 'environment_dogs',
 'environment.cats': 'environment_cats',
 'primary_photo_cropped.small': 'primary_photo_cropped_small',
 'primary_photo_cropped.medium': 'primary_photo_cropped_medium',
 'primary_photo_cropped.large': 'primary_photo_cropped_large',
 'primary_photo_cropped.full': 'primary_photo_cropped_full',
 'contact.email': 'contact_email',
 'contact.phone': 'contact_phone',
 'contact.address.address1': 'contact_address_address1',
 'contact.address.address2': 'contact_address_address2',
 'contact.address.city': 'contact_address_city',
 'contact.address.state': 'contact_address_state',
 'contact.address.postcode': 'contact_address_postcode',
 'contact.address.country': 'contact_address_country'})

In [9]:
animal3_df.head()

,id,organization_id,url,type,species,age,gender,size,coat,tags,...,contact_email,contact_phone,contact_address_address1,contact_address_address2,contact_address_city,contact_address_state,contact_address_postcode,contact_address_country,animal_id,animal_type
0,50404682,CA2331,https://www.petfinder.com/dog/mollie-50404682/...,Dog,Dog,Baby,Female,Large,Short,"[lab, golden]",...,iamk9connection@gmail.com,None,None,None,Carlsbad,CA,92009,US,50404682,dog
1,50404792,ME127,https://www.petfinder.com/dog/molly-barth-5040...,Dog,Dog,Adult,Female,Large,None,[],...,contactus@thepixelfund.org,None,None,None,Gorham,ME,04038,US,50404792,dog
2,50404733,LA48,https://www.petfinder.com/dog/elmer-50404733/l...,Dog,Dog,Senior,Male,Small,Long,[],...,aalslidell@gmail.com,985-646-4267,2700 Terrace Ave,None,Slidell,LA,70458,US,50404733,dog
3,50404777,NM198,https://www.petfinder.com/cat/paulo-50404777/n...,Cat,Cat,Baby,Male,Small,Short,[],...,corella@zianet.com,575-639-3036,2211 N. Mesquite,None,Las Cruces,NM,88001,US,50404777,cat
4,50404790,WV143,https://www.petfinder.com/cat/mo-50404790/wv/f...,Cat,Cat,Adult,Male,Small,None,[],...,adoptionisforever@pethelpersinc.org,(304) 672-7387,726 East Park Ave,Pmb 311,Fairmont,WV,26554,US,50404790,cat


In [10]:
organizations_df = pf.organizations(results_per_page=100, pages=100, return_df=True)

/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/petpy/api.py:1049: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  results_df = json_normalize(results[key])


In [11]:
# photos field is creating an error in pandas because it contains a list within the field. 
org_df = organizations_df.drop(columns=['photos'])

In [12]:
# change column header names to remove dot notation for mongo
org2_df = org_df.rename(columns = {
 'address.address1': "address_address1",
 'address.address2': "address_address2",
 'address.city': "address_city" ,
 'address.state': "address_state",
 'address.postcode': "address_postcode",
 'address.country': "address_country",
 'hours.monday': "hours_monday",
 'hours.tuesday': "hours_tuesday",
 'hours.wednesday': "hours_wednesday",
 'hours.thursday': "hours_thursday",
 'hours.friday': "hours_friday",
 'hours.saturday': "hours_saturday",
 'hours.sunday': "hours_sunday",
 'adoption.policy': "adoption_policy" ,
 'adoption.url': "adoption_url",
 'social_media.facebook': "socialmedia_facebook",
 'social_media.twitter': "socialmedia_twitter",
 'social_media.youtube': "socialmedia_youtube",
 'social_media.instagram': "socialmedia_instagram",
 'social_media.pinterest': "socialmedia_pinterest",
})

In [13]:
# now try to drop duplicates - based on all columns
org2_df.drop_duplicates()
# new length is 8932

,id,name,email,phone,url,website,mission_statement,distance,address_address1,address_address2,...,hours_saturday,hours_sunday,adoption_policy,adoption_url,socialmedia_facebook,socialmedia_twitter,socialmedia_youtube,socialmedia_instagram,socialmedia_pinterest,organization_id
0,PA912,The Shelter Animal Project,tsapadoption@gmail.com,None,https://www.petfinder.com/member/us/pa/fairles...,None,None,None,PO Box 24,None,...,None,None,None,None,None,None,None,None,None,pa912
1,GA762,Special Touch Animal Rescue,STARRadopt@aol.com,(404) 983-3581,https://www.petfinder.com/member/us/ga/acworth...,None,We are a private foster home group. We are NOT...,None,None,None,...,None,None,None,None,None,None,None,None,None,ga762
2,TX876,Gina's Heart of Gold Reptile Rescue,gdisteldorf@comcast.net,(713) 329-9304,https://www.petfinder.com/member/us/tx/houston...,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,tx876
3,OH749,Purr-fect Companions Sanctuary,purrfectcompanions@att.net,216-671-6369,https://www.petfinder.com/member/us/oh/lakewoo...,None,None,None,P.O.Box 770992,None,...,None,None,None,None,None,None,None,None,None,oh749
4,MO556,Montgomery City Animal Shelter/Project Preciou...,projectpreciouspaws@hotmail.com,1-573-310-4979,https://www.petfinder.com/member/us/mo/montgom...,None,None,None,915 Benton Street,None,...,None,None,None,None,None,None,None,None,None,mo556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9992,MI340,Cascades Humane Society,adopt@chspets.org,(517) 787-7837,https://www.petfinder.com/member/us/mi/jackson...,None,None,None,1515 Carmen Drive,None,...,12pm-6pm,Closed,None,None,https://www.facebook.com/chspets/,None,None,https://www.instagram.com/cascadeshumanesociet...,None,mi340
9993,NY186,Pet Adoption League,jjmjohnson27@aol.com,(914) 263-2967,https://www.petfinder.com/member/us/ny/yonkers...,http://www.petadoptleague.org,The Pet Adoption League (PAL) of Westchester i...,None,PO Box 914,None,...,None,None,None,None,https://www.facebook.com/PetAdoptionLeagueWest...,None,None,None,None,ny186
9994,IL823,Pets and Vets USA,petsandvetsusa@gmail.com,630-235-3777,https://www.petfinder.com/member/us/il/roselle...,None,All of our dogs work with a certified trainer...,None,25W571 Lake Street,None,...,None,None,Please email petsandvetsusa.com if you are int...,None,None,None,None,None,None,il823
9998,IL559,Clinton County Humane Society,ccilhs@ccilhs.org,618-526-4500,https://www.petfinder.com/member/us/il/breese/...,None,The Clinton County Humane Society (CCHS) is co...,None,1301 Apple Ln.,None,...,None,None,All of our dogs are in foster care and our ca...,None,None,None,None,None,None,il559


## Lat and Long DF and Add to Org Collection

In [14]:
zipfile_df = pd.read_csv("col_resources/zip_lat_lon.csv")

In [15]:
zipcode_df = zipfile_df[["Zip", "Latitude", "Longitude"]]

In [16]:
#need to convert datatypes in order to merge dataframes 
convert_zipdict = {'Zip': str}
zip_converted_df = zipcode_df.astype(convert_zipdict)

In [17]:
#need to convert datatypes in order to merge dataframes
convert_orgdict = {'address_postcode': str}
org_converted_df = org2_df.astype(convert_orgdict)

In [18]:
#rename field in zipcode to merge with org df 
new_zip_df = zip_converted_df.rename(columns={'Zip': 'address_postcode'})

In [19]:
#merge zip code and org dataframes - now every org should have a corresponding latlong
org_zip_df = pd.merge(new_zip_df, org_converted_df,  how= "inner", on=["address_postcode", "address_postcode"])

In [20]:
org_zip2_df = org_zip_df.drop(columns=['organization_id'])

In [21]:
# rename id column (with CAPS) as organization_id column
org_zip3_df = org_zip2_df.rename(columns={'id': 'organization_id'})

In [22]:
# convert datatypes in order to merge dataframes 
convert_animaldict = {'id': float}
animal_conv_df = animal3_df.astype(convert_animaldict)

In [23]:
animal_conv_df.head()

,id,organization_id,url,type,species,age,gender,size,coat,tags,...,contact_email,contact_phone,contact_address_address1,contact_address_address2,contact_address_city,contact_address_state,contact_address_postcode,contact_address_country,animal_id,animal_type
0,50404682.0,CA2331,https://www.petfinder.com/dog/mollie-50404682/...,Dog,Dog,Baby,Female,Large,Short,"[lab, golden]",...,iamk9connection@gmail.com,None,None,None,Carlsbad,CA,92009,US,50404682,dog
1,50404792.0,ME127,https://www.petfinder.com/dog/molly-barth-5040...,Dog,Dog,Adult,Female,Large,None,[],...,contactus@thepixelfund.org,None,None,None,Gorham,ME,04038,US,50404792,dog
2,50404733.0,LA48,https://www.petfinder.com/dog/elmer-50404733/l...,Dog,Dog,Senior,Male,Small,Long,[],...,aalslidell@gmail.com,985-646-4267,2700 Terrace Ave,None,Slidell,LA,70458,US,50404733,dog
3,50404777.0,NM198,https://www.petfinder.com/cat/paulo-50404777/n...,Cat,Cat,Baby,Male,Small,Short,[],...,corella@zianet.com,575-639-3036,2211 N. Mesquite,None,Las Cruces,NM,88001,US,50404777,cat
4,50404790.0,WV143,https://www.petfinder.com/cat/mo-50404790/wv/f...,Cat,Cat,Adult,Male,Small,None,[],...,adoptionisforever@pethelpersinc.org,(304) 672-7387,726 East Park Ave,Pmb 311,Fairmont,WV,26554,US,50404790,cat


## Merge DFs to One and Send to MongoDB

In [24]:
# all_df = pd.merge(animal_df, org2_df,  how= "inner", on=["organization_id", "organization_id"])
all_df = pd.merge(animal_conv_df, org_zip3_df, on=["organization_id", "organization_id"])

In [25]:
conn = mongo_conn
client = pymongo.MongoClient(conn)

In [26]:
db = client.rescue_angels_db

In [27]:
final_data_collection = db.final_data
#drop collection if it exists
final_data_collection.drop()


In [28]:
final_mongo_dict = all_df.to_dict("records")

In [29]:
db.final_data.insert_many(final_mongo_dict)